In [ ]:
model_name = "roberta-base"

plodv2_filtered_data = load_dataset("surrey-nlp/PLODv2-filtered")
sample_fraction = 0.25
num_train_sample = int(len(plodv2_filtered_data['train']) * sample_fraction)
num_val_sample = int(len(plodv2_filtered_data['validation']) * sample_fraction)
train_indices = random.sample(range(len(plodv2_filtered_data['train'])), num_train_sample)
val_indices = random.sample(range(len(plodv2_filtered_data['validation'])), num_val_sample)
plodv2_filtered_train_data = plodv2_filtered_data['train'].select(train_indices)
plodv2_filtered_val_data = plodv2_filtered_data['validation'].select(val_indices)
ds['train'] = concatenate_datasets([ds['train'], plodv2_filtered_train_data])
ds['validation'] = concatenate_datasets([ds['validation'], plodv2_filtered_val_data])

train_df = pd.DataFrame(ds['train'])
val_df = pd.DataFrame(ds['validation'])
test_df = pd.DataFrame(ds['test'])

train_df['tokens'] = train_df['tokens'].apply(tuple)
train_df['pos_tags'] = train_df['pos_tags'].apply(tuple)
train_df['ner_tags'] = train_df['ner_tags'].apply(tuple)
train_df.drop_duplicates(subset=['tokens', 'pos_tags', 'ner_tags'], inplace=True)
train_df['tokens'] = train_df['tokens'].apply(list)
train_df['pos_tags'] = train_df['pos_tags'].apply(list)
train_df['ner_tags'] = train_df['ner_tags'].apply(list)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
validation_dataset = Dataset.from_pandas(val_df)

dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "validation": validation_dataset
})


additional_dataset = dataset_dict

tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base",add_prefix_space=True)
label_all_tokens = True
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = additional_dataset.map(tokenize_and_align_labels, batched=True)
model = RobertaForTokenClassification.from_pretrained("roberta-base", num_labels=len(label_list))
args = TrainingArguments(
    model_name,
    eval_strategy= "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_strategy="steps",
    logging_steps=10,
    optim="adafactor",
    lr_scheduler_type="constant",
)
data_collator = DataCollatorForTokenClassification(tokenizer)
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
print(results)

In [ ]:
# Class wise bar chart
labels = [key for key in results if isinstance(results[key], dict)]  # e.g., ['AC', 'LF']
precision = [results[label]['precision'] for label in labels]
recall = [results[label]['recall'] for label in labels]
f1 = [results[label]['f1'] for label in labels]

x = range(len(labels))
width = 0.2

plt.figure(figsize=(8, 6))
plt.bar([p - width for p in x], precision, width, label='Precision')
plt.bar(x, recall, width, label='Recall')
plt.bar([p + width for p in x], f1, width, label='F1 Score')

plt.xticks(x, labels)
plt.ylim(0, 1)
plt.ylabel('Score')
plt.title('Token-level Metrics by Class')
plt.legend()
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

flat_true = [item for sublist in true_labels for item in sublist]
flat_pred = [item for sublist in true_predictions for item in sublist]
labels = sorted(set(flat_true + flat_pred))

cm = confusion_matrix(flat_true, flat_pred, labels=labels)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Confusion Matrix for PLOD-CW-25 + 25% PLODv2-filtered')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()
